# 여기서부터 순서대로 다 실행하기

In [1]:
!pip install google-auth
!pip install twilio

In [2]:
import os

json_file_path = '####.json'
print(os.path.exists(json_file_path))

import tkinter as tk
from tkinter import simpledialog, messagebox, Label, Entry, Button, Toplevel

import gspread
from google.oauth2.service_account import Credentials
from datetime import datetime

from twilio.rest import Client

True


In [3]:
scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
json = '####.json'
credentials = Credentials.from_service_account_file(json, scopes = scope)
gc = gspread.authorize(credentials)
sheet_url = 'https://docs.google.com/spreadsheets/d/1j1Md6qe-_tYfJ6NTIwxNQFw-9ohFYelyZBz8eWn0KbY/edit?hl=ko#gid=0'
doc = gc.open_by_url(sheet_url)
worksheet = doc.worksheet('DB')

In [4]:
class Queue:
    def __init__(self):
        self.queue = []
        scopes = ['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive']
        json = '####.json'  
        sheet_url = 'https://docs.google.com/spreadsheets/d/1j1Md6qe-_tYfJ6NTIwxNQFw-9ohFYelyZBz8eWn0KbY/edit?hl=ko#gid=0'
        worksheet = doc.worksheet('DB')
        credentials = Credentials.from_service_account_file(json, scopes=scopes)
        self.client = gspread.authorize(credentials)
        self.sheet = self.client.open_by_url(sheet_url).sheet1

    def enqueue(self, name, phone, people):
        col_values = len(worksheet.col_values(1)) + 1
        # 데이터를 큐에 추가
        self.queue.append((name, phone, people))
        # 바로 구글 시트에 데이터 저장
        self.sheet.insert_row([name, phone, people], col_values)
    
    def dequeue(self):
        if not self.is_empty():
            return self.queue.pop(0)
        else:
            raise IndexError("Queue is empty")

    def is_empty(self):
        return len(self.queue) == 0

    def size(self):
        return len(self.queue)

# 큐 및 구글 시트 매니저 초기화
DB = Queue()

In [10]:
def get_user_input():
    while True:
        name = input("이름을 입력하세요 (종료하려면 빈 문자열 입력): ")
        if name == "":
            break
        while True:
            phone_number = input("전화번호를 입력하세요 (11자리 숫자): ")
            if phone_number.isdigit() and len(phone_number) == 11:
                break
            print("전화번호는 11자리 숫자여야 합니다. 다시 입력해주세요.")
        count = int(input("인원 수를 입력하세요: "))
        DB.enqueue(name, phone_number, count)


user_entries = get_user_input()

이름을 입력하세요 (종료하려면 빈 문자열 입력):  


# 학생 정보 입력 받기

In [7]:
def get_user_input():
    root = tk.Tk()
    root.withdraw()  # 메인 윈도우 숨기기

    while True:
        name = simpledialog.askstring("입력", "이름을 입력하세요 (종료하려면 빈 문자열 입력):", parent=root)
        if name == "":
            break
        while True:
            phone_number = simpledialog.askstring("입력", "전화번호를 입력하세요 (11자리 숫자만 only number):", parent=root)
            if phone_number.isdigit() and len(phone_number) == 11:
                break  # 전화번호가 조건에 맞으면 while 루프를 빠져나옵니다.
            messagebox.showerror("입력 오류", "전화번호는 11자리 숫자여야 합니다. 다시 입력해주세요.", parent=root)
        count = simpledialog.askinteger("입력", "인원 수를 입력하세요:", parent=root)

        DB.enqueue(name, phone_number, count)
        print(f"입력된 정보: {name}, {phone_number}, {count}")

get_user_input()

AttributeError: 'NoneType' object has no attribute 'isdigit'

# 문자보내기

In [8]:
def send_sms():
    root = tk.Tk()
    root.withdraw()

    while True:
        seq = simpledialog.askinteger("입력", "몇 번째 학생에게 문자를 보낼까요? : [엑셀 row값]", parent=root)
        if seq is None:  # 사용자가 취소 버튼을 눌렀을 때
            messagebox.showinfo("알림", "작업이 취소되었습니다.")
            break
        name = worksheet.cell(seq, 1).value
        phone_number = worksheet.cell(seq, 2).value
        if phone_number.startswith('0'):
            phone_number = '+82' + phone_number[1:]
        people = worksheet.cell(seq, 3).value
        
        account_sid = '##'
        auth_token = '##'
        client = Client(account_sid, auth_token)
    
        try:
            message = client.messages.create(
                body=f'{name}님 기다리느라 고생하셨어요! 지금 [유영 포토부스]에 입장해주세요. 해당문자를 보여주시면 안내 도와드리겠습니다',
                from_='+15307231735',
                to=phone_number
            )
            status = "전송완료"
            messagebox.showinfo("전송 완료", f"{phone_number} 번호로 {name}에게 {people}명에게 문자 전송 완료")
        except Exception as e:
            status = "실패"
            print ("전송실패")
            print(f"Failed to send message: {str(e)}")
    
        timestamp = datetime.now().strftime("%H시 %M분 %S초")
        worksheet.update_cell(seq, 5, status)
        worksheet.update_cell(seq, 6, timestamp)
        continue_answer = messagebox.askyesno("계속", "다른 학생에게도 문자를 보내시겠습니까?")
        if not continue_answer:
            break
            
    root.destroy()
send_sms()